In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import shap
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate, cross_val_score, cross_val_predict
from sklearn.metrics import matthews_corrcoef, confusion_matrix, roc_auc_score, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline
from sklearn.metrics import recall_score, accuracy_score
from sklearn.impute import KNNImputer, SimpleImputer

In [2]:
random_state = 7
cv_method = StratifiedKFold(n_splits=5)

# Training of tsfresh-selected Training Max 2 years and eGFR 6 times

In [3]:
df = pd.read_csv('../data/train_all_2_6.gz')
df.shape

(4965, 49610)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4965 entries, 0 to 4964
Columns: 49610 entries, id to cat8
dtypes: float64(49588), int64(2), object(20)
memory usage: 1.8+ GB


In [5]:
df['aki_1'].fillna(value=0, inplace=True)
df['aki_2'].fillna(value=0, inplace=True)
df['aki_3'].fillna(value=0, inplace=True)

In [6]:
df['gn'].value_counts()

no                 4508
igan                184
anca                 85
lupus nephritis      45
membranous           36
unspecific           29
fsgs                 27
mcd                  24
mp/mcgn              19
fgn/itg               6
pign                  2
Name: gn, dtype: int64

In [7]:
df.loc[df.gn == 'pign', 'gn'] = 'unspecific'
df.loc[df.gn == 'fgn/itg', 'gn'] = 'unspecific'

In [8]:
percent_missing = df.isnull().sum()*100 / len(df)
missing_df = pd.DataFrame({'column_name': df.columns, 'percent_miss': percent_missing})

In [9]:
missing_df.sort_values(by='percent_miss', ascending=False)

,column_name,percent_miss
"sta_sys__matrix_profile__feature_""""""""min""""""""__threshold_0.98","sta_sys__matrix_profile__feature_""""""""min""""""""__...",100.0
"urate__fft_coefficient__attr_""""""""real""""""""__coeff_82","urate__fft_coefficient__attr_""""""""real""""""""__coe...",100.0
"urate__fft_coefficient__attr_""""""""real""""""""__coeff_62","urate__fft_coefficient__attr_""""""""real""""""""__coe...",100.0
"urate__fft_coefficient__attr_""""""""real""""""""__coeff_63","urate__fft_coefficient__attr_""""""""real""""""""__coe...",100.0
"urate__fft_coefficient__attr_""""""""real""""""""__coeff_64","urate__fft_coefficient__attr_""""""""real""""""""__coe...",100.0
...,...,...
htn,htn,0.0
aki_3,aki_3,0.0
aki_2,aki_2,0.0
aki_1,aki_1,0.0


### Number of patients reaching ESKD within training period 

In [10]:
df['cat2'].value_counts()

non_eskd    3708
eskd        1257
Name: cat2, dtype: int64

In [11]:
df.set_index('id', inplace=True)

In [12]:
def to_category(df):
    cols = df.select_dtypes(include='object').columns
    for col in cols:
        df[col] = df[col].astype('category')
    return df

def drop_missing(df):
    threshold = len(df)*0.6
    df.dropna(axis=1, thresh=threshold, inplace=True)
    return df

def copy_df(df):
    return df.copy()

In [13]:
df_cleaned = (df.pipe(copy_df).pipe(drop_missing).pipe(to_category))
df_cleaned.shape

(4965, 7781)

In [14]:
df_rm_intrain = df_cleaned.query("cat2 == 'non_eskd'")
df_rm_intrain.shape

(3708, 7781)

In [15]:
df_rm_intrain.to_csv('../data/cleaned60_all_2_6.gz', index=True, compression='gzip')

In [16]:
df_rm_intrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3708 entries, 3916 to 23864
Columns: 7781 entries, ureacreat_ratio__variance_larger_than_standard_deviation to cat8
dtypes: category(20), float64(7760), int64(1)
memory usage: 219.7 MB


In [27]:
df_rm_intrain = (df_rm_intrain.pipe(copy_df).pipe(to_category))
df_rm_intrain.shape

,ureacreat_ratio__variance_larger_than_standard_deviation,ureacreat_ratio__has_duplicate_max,ureacreat_ratio__has_duplicate_min,ureacreat_ratio__has_duplicate,ureacreat_ratio__sum_values,ureacreat_ratio__abs_energy,ureacreat_ratio__mean_abs_change,ureacreat_ratio__mean_change,ureacreat_ratio__mean_second_derivative_central,ureacreat_ratio__median,...,cat3.5,cat4,cat4.5,cat5,cat5.5,cat6,cat6.5,cat7,cat7.5,cat8
id,,,,,,,,,,,,,,,,,,,,,
3916,False,False,False,False,0.152681,0.007990,0.016659,0.004164,-0.016659,0.049505,...,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd
3918,False,False,False,False,0.201586,0.013588,0.004620,-0.004620,-0.000227,0.067347,...,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd
3921,False,False,False,False,0.245740,0.021038,0.020792,0.020792,-0.008116,0.087324,...,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd
3924,False,False,False,False,0.185151,0.011960,0.015921,0.015921,0.006274,0.057534,...,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd
3930,False,False,False,False,0.496335,0.035761,0.011239,0.003173,0.002983,0.069399,...,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23771,False,False,False,False,0.160833,0.009284,0.024167,0.011667,0.024167,0.050000,...,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd
23775,False,False,False,False,0.044286,0.001961,NaN,NaN,NaN,0.044286,...,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd
23780,False,False,False,False,0.078750,0.006202,NaN,NaN,NaN,0.078750,...,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd,non_eskd


### Number of patients reaching ESKD in at 1y post-training 

In [17]:
df_rm_intrain['cat3'].value_counts()

non_eskd    3586
eskd         122
Name: cat3, dtype: int64

### Number of patients reaching ESKD in 2y post-training

In [18]:
df_rm_intrain['cat4'].value_counts()

non_eskd    3484
eskd         224
Name: cat4, dtype: int64

### Number of patients reaching ESKD in 5y post-training

In [19]:
df_rm_intrain['cat7'].value_counts()

non_eskd    3261
eskd         447
Name: cat7, dtype: int64

In [20]:
percent_missing = df_rm_intrain.isnull().sum()*100 / len(df_rm_intrain)
missing_df_rm_intrain = pd.DataFrame({'column_name': df_rm_intrain.columns, 'percent_miss': percent_missing})

In [21]:
missing_df_rm_intrain.sort_values(by='percent_miss', ascending=False)

,column_name,percent_miss
"chloride__augmented_dickey_fuller__attr_""""""""teststat""""""""__autolag_""""""""AIC""""""""","chloride__augmented_dickey_fuller__attr_""""""""te...",51.024811
chloride__partial_autocorrelation__lag_1,chloride__partial_autocorrelation__lag_1,51.024811
chloride__autocorrelation__lag_3,chloride__autocorrelation__lag_3,51.024811
"chloride__augmented_dickey_fuller__attr_""""""""pvalue""""""""__autolag_""""""""AIC""""""""","chloride__augmented_dickey_fuller__attr_""""""""pv...",51.024811
"chloride__fft_coefficient__attr_""""""""abs""""""""__coeff_2","chloride__fft_coefficient__attr_""""""""abs""""""""__c...",50.970874
...,...,...
dkd,dkd,0.000000
htn,htn,0.000000
aki_3,aki_3,0.000000
aki_2,aki_2,0.000000


## Features and Target Input for Training 

In [22]:
dropped_cols = list(df_rm_intrain.columns[-18:])
dropped_cols.remove('age.init')
dropped_cols

['egfr.y',
 'cat0.5',
 'cat1',
 'cat1.5',
 'cat2',
 'cat2.5',
 'cat3',
 'cat3.5',
 'cat4',
 'cat4.5',
 'cat5',
 'cat5.5',
 'cat6',
 'cat6.5',
 'cat7',
 'cat7.5',
 'cat8']

In [23]:
X = df_rm_intrain.drop(dropped_cols,axis=1).copy()
y = df_rm_intrain['cat7']
X.shape, y.shape

((3708, 7764), (3708,))

In [24]:
y_mapped = y.map({'non_eskd':'no', 'eskd':'yes'})
y_mapped

id
3916     no
3918     no
3921     no
3924     no
3930     no
         ..
23771    no
23775    no
23780    no
23823    no
23864    no
Name: cat7, Length: 3708, dtype: category
Categories (2, object): ['yes', 'no']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y_mapped, test_size=0.3, stratify=y_mapped, random_state=random_state)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2595, 7764), (1113, 7764), (2595,), (1113,))

In [26]:
lab_encode = LabelEncoder()
y_train = lab_encode.fit_transform(y_train)
y_test = lab_encode.transform(y_test)
y_train.shape, y_test.shape

((2595,), (1113,))

In [27]:
unique_values, counts = np.unique(y_train, return_counts=True)
ratio = dict(zip(unique_values, counts))
ratio

{0: 2282, 1: 313}

In [28]:
imbalance_ratio = ratio[0] / ratio[1]
imbalance_ratio

7.2907348242811505

In [29]:
category_cols = list(X_train.select_dtypes(include='category').columns)
numeric_cols = list(X_train.select_dtypes(include=['int', 'float']).columns)
one_hot = OneHotEncoder(handle_unknown='ignore')
num_imputer = KNNImputer(n_neighbors=5)
len(category_cols), len(numeric_cols)

(4, 7760)

In [30]:
preprocess = ColumnTransformer([('num_imputing', num_imputer, numeric_cols), ('cat_encoder', one_hot, category_cols)], remainder='passthrough')
X_train_processed = preprocess.fit_transform(X_train)
X_train_processed.shape

(2595, 7775)

In [31]:
X_test_processed = preprocess.transform(X_test)

In [32]:
xgb_class = xgb.XGBClassifier(n_jobs=-1, random_state = random_state, n_estimators=1000, use_label_encoder=False, eval_metric='logloss', scale_pos_weight=imbalance_ratio)
sampler = SMOTETomek(sampling_strategy=0.5, random_state=random_state)
steps = [('resampling', sampler), ('model', xgb_class)]
pipeline = Pipeline(steps=steps)

In [33]:
pipeline.fit(X_train_processed, y_train)

Pipeline(steps=[('resampling',
                 SMOTETomek(random_state=7, sampling_strategy=0.5)),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eval_metric='logloss',
                               gamma=0, gpu_id=-1, importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=1000,
                               n_jobs=-1, num_parallel_tree=1, random_state=7,
                               reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=7.2907348242811505, subsample=1,
                               tree_method='exact', use_label_encoder=False,
    

In [34]:
y_pred = pipeline.predict(X_test_processed)

In [35]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[936,  43],
       [ 79,  55]])

In [36]:
roc_auc_score(y_test, y_pred)

0.6832626957144817

In [37]:
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94       979
           1       0.56      0.41      0.47       134

    accuracy                           0.89      1113
   macro avg       0.74      0.68      0.71      1113
weighted avg       0.88      0.89      0.88      1113



In [38]:
print(matthews_corrcoef(y_test, y_pred))

0.4209221231531018


In [39]:
importance = pipeline.named_steps['model'].feature_importances_
importance.shape, type(importance)

((7775,), numpy.ndarray)

In [40]:
category_label = list(preprocess.named_transformers_['cat_encoder'].get_feature_names(category_cols))
feature_label = numeric_cols + category_label
len(feature_label)

7775

In [41]:
pd.set_option('display.max_rows', 400)
eval_df = pd.DataFrame({'label': feature_label, 'importance_value': importance})
eval_df.sort_values(by='importance_value', ascending=False).head(100)

,label,importance_value
1815,creatinine__maximum,0.085064
1792,creatinine__median,0.035147
4901,"ggt__change_quantiles__f_agg_""""""""var""""""""__isab...",0.015721
2875,phosphate__large_standard_deviation__r_0.35000...,0.015622
1071,calcium__symmetry_looking__r_0.30000000000000004,0.013484
6774,mcv__cwt_coefficients__coeff_0__w_2__widths_(2...,0.011508
6268,mch__time_reversal_asymmetry_statistic__lag_3,0.010702
6772,mcv__index_mass_quantile__q_0.8,0.010090
5785,eosinophils__ratio_beyond_r_sigma__r_0.5,0.010007
6691,mcv__first_location_of_minimum,0.009657


In [42]:
xgb_class = xgb.XGBClassifier(n_jobs=-1, random_state = random_state, n_estimators=1000, use_label_encoder=False, eval_metric='logloss', scale_pos_weight=imbalance_ratio)
sampler = SMOTETomek(random_state=random_state)
steps = [('resampling', sampler), ('model', xgb_class)]
pipeline = Pipeline(steps=steps)

In [43]:
pipeline.fit(X_train_processed, y_train)

Pipeline(steps=[('resampling', SMOTETomek(random_state=7)),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eval_metric='logloss',
                               gamma=0, gpu_id=-1, importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=1000,
                               n_jobs=-1, num_parallel_tree=1, random_state=7,
                               reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=7.2907348242811505, subsample=1,
                               tree_method='exact', use_label_encoder=False,
                               validate_para

In [44]:
y_pred = pipeline.predict(X_test_processed)

In [45]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[923,  56],
       [ 74,  60]])

In [46]:
roc_auc_score(y_test, y_pred)

0.6952799841446495

In [47]:
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.93      0.94      0.93       979
           1       0.52      0.45      0.48       134

    accuracy                           0.88      1113
   macro avg       0.72      0.70      0.71      1113
weighted avg       0.88      0.88      0.88      1113

